# Building Firmware

In [ ]:
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'NONE'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/your_project
make PLATFORM=$1 CRYPTO_TARGET=$2

# Capture

## Setup

In [ ]:
import chipwhisperer as cw
scope = cw.scope()
target = cw.target(scope)
scope.default_setup()

In [ ]:
fw_path = "../../hardware/victims/firmware/path/to/binary"
prog = cw.programmers.STM32FProgrammer
#prog = cw.programmers.XMEGAProgrammer

In [ ]:
cw.program_target(scope, prog, fw_path)

## Create Project

In [ ]:
project = cw.create_project("projects/project_name.cwp", overwrite=True)

## Basic Capture Loop (For Analyzer)

In [ ]:
#Capture Traces
from tqdm import tqdm
import numpy as np
import time

ktp = cw.ktp.Basic(target=target)

N = 50  # Number of traces
for i in tqdm(range(N), desc='Capturing traces'):

    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    ret = cw.capture_trace(scope, target, text, key)
    if not ret:
        continue
    wave, textout = ret
    project.traces.append((wave, text, textout, key))

## Basic Capture Loop (No Project)

In [ ]:
#Capture Traces
from tqdm import tqdm
import numpy as np
import time

ktp = cw.ktp.Basic(target=target)

traces = []
textin = []
keys = []
N = 50  # Number of traces
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    textin.append(text)
    keys.append(key)
    ret = cw.capture_trace(scope, target, text, key)
    if not ret:
        continue
    wave, textout = ret
    traces.append(wave)

#Convert traces to numpy arrays
trace_array = np.asarray(traces)  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray(textin)
known_keys = np.asarray(keys)  # for fixed key, these keys are all the same

## Multiple Trace Capture

Captures multiple traces with the same plaintext and stitches them together. This is helpful for algorithms with very long traces (such as RSA)

In [ ]:
def run_multi_trace(scope, target, start, step, num_step):
    scope.adc.samples = step
    num_stepped = 0
    trace = np.zeros(step*num_step)
    while num_stepped < num_step:

        scope.adc.offset = start + (step * num_stepped)

        # run aux stuff that should run before the scope arms here
        scope.arm()

        # run aux stuff that should run after the scope arms here

        target.go()
        # wait for target to finish
        while target.isDone() is False:
            timeout -= 1
            time.sleep(0.01)

        try:
            ret = scope.capture()
            if ret:
                print('Timeout happened during acquisition')
        except IOError as e:
            print('IOError: %s' % str(e))

        # run aux stuff that should happen after trace here

        #_ = target.readOutput() # throw out the target response
        # append the new trace
        new_trace = scope.getLastTrace()
        if len(new_trace) < step:
            new_trace = np.zeros(step)
        trace[((num_stepped) * step):((num_stepped+1) * step)] = new_trace
        num_stepped += 1
    return trace

## Saving Project Traces

In [ ]:
project.save()

## Disconnect

In [ ]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

# Analysis (With Analyzer)

## Loading Project

In [ ]:
import chipwhisperer as cw
import chipwhisperer.analyzer as cwa
project = cw.open_project("projects/Your project")

## Setting up Attack

In [ ]:
import chipwhisperer as cw
attack = cw.cpa(project, cw.leakage_models.sbox_output)

## Attack With Table

In [ ]:
cb = cw.get_jupyter_callback(attack)
attack_results = attack.run(cb)

## Plot Data

In [ ]:
plot_data = cw.analyzer_plots(attack_results)

In [ ]:
rets = []
for i in range(16):
    rets.append(plot_data.outputVsTime(i))

In [ ]:
for bnum in range(16):
    ret = plot_data.pgeVsTrace(bnum)

In [ ]:
for bnum in range(16):
    ret = plot_data.corrVsTrace(bnum)

# Plotting With Bokeh

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()
p.line(xdata, ydata, line_color="red")
show(p)

# Other

## Serial Read

In [ ]:
response = target.read()

## Resetter

In [ ]:
import time
def reset_target(scope):
    scope.io.nrst = 'low'
    #scope.io.pdic = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high'
    #scope.io.pdic = 'high'